In [2]:
%reload_ext autoreload
%autoreload 2

from preprocessing_merge import *
from preprocessing_train_test import preprocess_train_test
# from preprocessing import *

pd.set_option('display.max_columns', 100)  # 设置显示100列
pd.set_option('display.max_rows', 100)   # 设置显示100行

In [3]:
%%time

# Import data
train_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "test.csv"))

commerical_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "auxiliary-data/sg-commerical-centres.csv"))
market_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "auxiliary-data/sg-gov-markets-hawker-centres.csv"))

population_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "auxiliary-data/sg-population-demographics.csv"))

primary_School_df =  pd.read_csv(os.path.join(DATA_DIRECTORY, "auxiliary-data/sg-primary-schools.csv"))
secondary_School_df =  pd.read_csv(os.path.join(DATA_DIRECTORY, "auxiliary-data/sg-secondary-schools.csv"))

mall_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "auxiliary-data/sg-shopping-malls.csv"))
train_station_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "auxiliary-data/sg-train-stations.csv"))

CPU times: user 703 ms, sys: 102 ms, total: 805 ms
Wall time: 817 ms


In [88]:
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import skew 
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import xgboost as xgb

In [7]:
X=train_df.drop(['resale_price'],axis=1)
y=train_df['resale_price']
train_X, validation_X, train_y, validation_y = train_test_split(X, y, test_size=0.2)

In [11]:
train = train_X
train['resale_price'] = train_y
validation = validation_X
print(train.shape)
print(test.shape)

(345385, 17)
(86347, 16)


In [14]:
train_processed, validation_processed = preprocess_train_test(train, validation)
print(train_processed.shape)
print(validation_processed.shape)

(345375, 44)
(86347, 43)


In [17]:
train_gdf = merge_auxiliary_data(train_processed, commerical_df, market_df, population_df, primary_School_df, secondary_School_df, mall_df, train_station_df)
validation_gdf = merge_auxiliary_data(validation_processed, commerical_df, market_df, population_df, primary_School_df, secondary_School_df, mall_df, train_station_df)

print(train_gdf.shape)
print(validation_gdf.shape)

(345375, 82)
(86347, 81)


In [62]:
train_gdf_bak = train_gdf
validation_gdf_bak = validation_gdf

In [63]:

def init_check(df):
    """
    A function to make initial check for the dataset including the name, data type, 
    number of null values and number of unique varialbes for each feature.
    
    Parameter: dataset(DataFrame)
    Output : DataFrame
    """
    columns = df.columns    
    lst = []
    for feature in columns : 
        dtype = df[feature].dtypes
        num_null = df[feature].isnull().sum()
        num_unique = df[feature].nunique()
        lst.append([feature, dtype, num_null, num_unique])
    
    check_df = pd.DataFrame(lst)
    check_df.columns = ['feature','dtype','num_null','num_unique']
    # check_df = check_df.sort_values(by='dtype', axis=0, ascending=True)
    
    return check_df

pd.set_option('display.max_rows', None)


init_check(train_gdf_bak)

,feature,dtype,num_null,num_unique
0,town,object,0,26
1,flat_type,object,0,7
2,block,object,0,2471
3,street_name,object,0,1103
4,storey_range,object,0,25
5,floor_area_sqm,float64,0,184
6,flat_model,object,0,20
7,eco_category,object,0,1
8,lease_commence_date,int64,0,54
9,elevation,float64,0,1


In [64]:
init_check(validation_gdf_bak)

,feature,dtype,num_null,num_unique
0,town,object,0,26
1,flat_type,object,0,7
2,block,object,0,2420
3,street_name,object,0,1093
4,storey_range,object,0,24
5,floor_area_sqm,float64,0,168
6,flat_model,object,0,20
7,eco_category,object,0,1
8,lease_commence_date,int64,0,53
9,elevation,float64,0,1


In [65]:
train_gdf_bak = train_gdf_bak.drop(columns={'eco_category', 'elevation', 'geometry', 'street_name', 'block', 'planning_area'}, axis=1)
validation_gdf_bak = validation_gdf_bak.drop(columns={'eco_category', 'elevation', 'geometry', 'street_name', 'block', 'planning_area'}, axis=1)

In [66]:
train_gdf_bak = train_gdf_bak.drop(columns={'grid_price', 'grid_price_psm', 'opening_year'}, axis=1)
validation_gdf_bak = validation_gdf_bak.drop(columns={'grid_price', 'grid_price_psm', 'opening_year'}, axis=1)

In [67]:
train_y = train_gdf_bak['resale_price']
train_features = train_gdf_bak.drop(['resale_price'], axis=1)
validation_features = validation_gdf_bak
features = pd.concat([train_features, validation_features]).reset_index(drop=True)

In [68]:
init_check(features)

,feature,dtype,num_null,num_unique
0,town,object,0,26
1,flat_type,object,0,7
2,storey_range,object,0,25
3,floor_area_sqm,float64,0,187
4,flat_model,object,0,20
5,lease_commence_date,int64,0,54
6,subzone,object,0,155
7,region,object,0,5
8,rebased_month,int64,0,251
9,remaining_lease,int64,0,57


In [69]:
features = features.drop(['town', 'mrt_name'], axis=1)

In [70]:
objects = []
for i in features.columns:
    if features[i].dtype == object:
        objects.append(i)

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics.append(i)
features.update(features[numerics])

In [71]:
objects

['flat_type',
 'storey_range',
 'flat_model',
 'subzone',
 'region',
 'codes',
 'mrt_type',
 'codes_name',
 'type_commerical']

In [72]:
encoder = LabelEncoder()
labelFeatures = ['storey_range', 'flat_model', 'subzone', 'codes', 'codes_name', 'flat_type']
for each in labelFeatures:
    features[each] = encoder.fit_transform(features[each])

In [73]:
skew_features = features[numerics].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 1.0] # 0.5

print("high_skew: ", high_skew)
skew_index = high_skew.index

for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))

high_skew:  flat_model_psm               10.419092
mrt_lrt_links                 8.530379
storey_range_price_psm        5.513750
storey_range_price            4.433453
flat_type_psm                 3.403469
distance_to_mrt_bins          2.928533
mrt_interchange_flag          2.674936
mrt_interchange_count         2.674936
street_name_price_psm         2.658701
MinSecDist                    2.270065
MinPrimaryDist                2.115721
subzone_price_psm             1.873654
adult_children_ratio_bins     1.861391
senior_citizen_count          1.827685
flat_model_price              1.814234
young_adult_count             1.790040
nearest_dist_market           1.650604
female_count                  1.620981
street_name_price             1.614855
population_count              1.582602
adult_count                   1.552388
male_count                    1.543005
teenager_count                1.437612
population_bins               1.425300
storey_range_start            1.281766
storey_range_

In [74]:
features.head()

,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,subzone,region,rebased_month,remaining_lease,1-room,2-room,3-room,4-room,5-room,executive,multi-generation,flat_type_price,flat_type_psm,flat_type_number,storey_range_start,storey_range_processed,storey_range_price,storey_range_price_psm,flat_model_price,flat_model_psm,subzone_price,subzone_price_psm,street_name_price,street_name_price_psm,planning_area_price,planning_area_price_psm,region_price,region_price_psm,codes,mrt_type,distance_to_mrt,distance_to_mrt_km,mrt_lrt_links,mrt_interchange_flag,mrt_interchange_count,codes_name,distance_to_mrt_bins,population_count,adult_count,children_count,senior_citizen_count,teenager_count,young_adult_count,female_count,male_count,male_female_ratio,male_female_ratio_bins,adult_children_ratio,adult_children_ratio_bins,population_bins,type_commerical,nearest_dist_commerical,inRangeCount_commerical,nearest_dist_market,inRangeCount_market,primary_id,MinPrimaryDist,nearPrimaryCount,second_id,MinSecDist,nearSecondCount,mall_id,MinShopDist,nearShopCount,has_wikilink
0,4,4,124.0,4,1996,88,north region,181,80,0,0,0,0,1,0,0,381856.054046,0.28321,5,3.497486,1.556986,0.118034,0.257081,0.118034,0.250444,0.118034,0.39473,0.118034,0.442253,277248.788527,0.329564,271182.180785,2679.147847,87,mrt,0.004821,0.459037,0.0,0.0,0.0,6,0.0,36.410674,31.029421,20.130259,12.775163,22.271002,20.660137,29.942719,30.530996,1.003950,2,2.649419,0.0,0.000000,CR,0.901890,2,0.395138,1,175,0.250224,6,45,0.157713,3,97,0.273935,5,0
1,3,0,104.0,12,2000,142,north region,232,80,0,0,0,1,0,0,0,297555.011428,0.28321,4,0.817346,0.734185,0.118034,0.257081,0.118034,0.250444,0.118034,0.39473,0.118034,0.442253,277248.788527,0.329564,271182.180785,2679.147847,87,mrt,0.002296,0.233653,0.0,0.0,0.0,6,0.0,45.609555,39.504726,26.772301,15.015204,27.407072,24.896471,37.508431,38.332580,1.003986,2,2.572469,0.0,1.273151,CR,1.937493,2,0.231060,1,66,0.107180,5,138,0.302748,2,98,0.197732,3,0
2,3,4,100.0,7,1998,142,north region,123,87,0,0,0,1,0,0,0,297555.011428,0.28321,4,3.497486,1.556986,0.118034,0.257081,0.118034,0.250444,0.118034,0.39473,0.118034,0.442253,277248.788527,0.329564,271182.180785,2679.147847,87,mrt,0.003833,0.371792,0.0,0.0,0.0,6,0.0,45.609555,39.504726,26.772301,15.015204,27.407072,24.896471,37.508431,38.332580,1.003986,2,2.572469,0.0,1.273151,CR,1.328432,2,0.314134,1,176,0.141061,6,138,0.169672,3,97,0.386568,3,0
3,3,2,102.0,7,1997,142,north region,63,91,0,0,0,1,0,0,0,297555.011428,0.28321,4,2.390899,1.204077,0.118034,0.257081,0.118034,0.250444,0.118034,0.39473,0.118034,0.442253,277248.788527,0.329564,271182.180785,2679.147847,87,mrt,0.004709,0.447698,0.0,0.0,0.0,6,0.0,45.609555,39.504726,26.772301,15.015204,27.407072,24.896471,37.508431,38.332580,1.003986,2,2.572469,0.0,1.273151,IEBP,1.263200,2,0.407054,1,134,0.166350,5,0,0.242044,2,112,0.253177,2,0
4,4,5,121.0,4,1998,142,north region,23,96,0,0,0,0,1,0,0,381856.054046,0.28321,5,4.396506,1.842451,0.118034,0.257081,0.118034,0.250444,0.118034,0.39473,0.118034,0.442253,277248.788527,0.329564,271182.180785,2679.147847,87,mrt,0.004575,0.437404,0.0,0.0,0.0,6,0.0,45.609555,39.504726,26.772301,15.015204,27.407072,24.896471,37.508431,38.332580,1.003986,2,2.572469,0.0,1.273151,CR,1.185093,2,0.386213,1,52,0.069914,6,45,0.150125,4,112,0.446556,4,0


In [75]:
final_features = pd.get_dummies(features).reset_index(drop=True)
final_features.shape

(431722, 79)

In [76]:
overfit = []
for i in final_features.columns:
    counts = final_features[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(final_features) * 100 > 99.94:
        overfit.append(i)

overfit = list(overfit)
final_features = final_features.drop(overfit, axis=1)

In [78]:
train_X = final_features.iloc[:len(train_features), :]
validation_X = final_features.iloc[len(train_features):, :]
train_X.shape, train_y.shape, validation_X.shape, validation_y.shape

((345375, 72), (345375,), (86347, 72), (86347,))

In [98]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.ensemble import GradientBoostingRegressor

from sklearn.pipeline import make_pipeline
# from lightgbm import LGBMRegressor


##  Error Metric

In [90]:
def evaluation(y, predictions):
    mae = mean_absolute_error(y, predictions)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    r_squared = r2_score(y, predictions)
    return mae, mse, rmse, r_squared

## XGBRegressor

In [93]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, random_state =7, nthread = -1)
model_xgb.fit(train_X, train_y)
predictions = model_xgb.predict(validation_X)

mae, mse, rmse, r_squared = evaluation(validation_y, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)


MAE: 140617.14686523742
MSE: 32955869982.175976
RMSE: 181537.51673462975
R2 Score: -0.9534991521597354
------------------------------


## GradientBoostingRegressor

In [99]:
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
gbr.fit(train_X, train_y)
predictions = gbr.predict(validation_X)

mae, mse, rmse, r_squared = evaluation(validation_y, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)

MAE: 140678.14731382215
MSE: 33014318490.117943
RMSE: 181698.4273187799
R2 Score: -0.956963758336765
------------------------------


## LGBMRegressor

In [94]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )
lightgbm.fit(train_X, train_y)
predictions = lightgbm.predict(validation_X)

mae, mse, rmse, r_squared = evaluation(validation_y, predictions)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r_squared)
print("-"*30)
rmse_cross_val = rmse_cv(lin_reg)
print("RMSE Cross-Validation:", rmse_cross_val)

NameError: name 'LGBMRegressor' is not defined

In [ ]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))
